In [8]:
import numpy as np
import pandas as pd

def which(self):
    try:
        self = list(iter(self))
    except TypeError as e:
        raise Exception("""'which' method can only be applied to iterables.
        {}""".format(str(e)))
    indices = [i for i, x in enumerate(self) if bool(x) == True]
    return(indices)

cb = pd.read_csv("codebook-reworked.csv")
vartype = cb.vartype
vartype = vartype[which(vartype != 0)]
index = 0
numeric = []
categorical = []
for i in vartype:
    if i in [1,2,4,5]:
        numeric.append(index)
    if i == 3:
        categorical.append(index)
    index = index + 1

In [9]:
df = pd.read_csv("train.csv")
group = df["personid"]
df = df.drop(['uniqueid', 'personid'], axis=1)
df = df.dropna(axis=1, how='all')

X = df.drop(["health"], axis=1)
colnames_train = X.columns

test = pd.read_csv("test.csv")
test = test.dropna(axis=1, how='all')
uniqueid = test['uniqueid']
test = test.drop(['uniqueid', 'personid'], axis=1)
colnames_test = test.columns

rm_train = set(colnames_train) - set(colnames_test)
rm_test = set(colnames_test) - set(colnames_train)

X = X.drop(list(rm_train), axis=1)
test = test.drop(list(rm_test), axis=1)

y = df["health"]

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

numeric_pipe = Pipeline([('imputer', SimpleImputer(strategy='mean')), ('scaler', MinMaxScaler())])
categorical_pipe = Pipeline([('imputer', SimpleImputer(strategy='constant', fill_value=9999999)), ('ohe', OneHotEncoder(handle_unknown = "ignore"))])
t = [('cat', categorical_pipe, categorical), ('num', numeric_pipe, numeric)]
col_transform = ColumnTransformer(transformers=t)

In [11]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(random_state=20803652, learning_rate_init=0.01, alpha=0.4, hidden_layer_sizes=1371, max_iter=1000)
mlp_pipeline = Pipeline(steps=[('prep',col_transform), ('m', model)])

fit = mlp_pipeline.fit(X, y)
predictions = fit.predict_proba(test)

In [12]:
# creating a submission file for best neural network model.

pred = pd.DataFrame(predictions)
df.insert(loc=0, column='uniqueid', value=uniqueid)
pred.to_csv("submission-neural.csv", index=False)